In [1]:
import import_ipynb #allows access to import other notebooks
from PreprocessLib import cleanText, removeSingleLists

importing Jupyter notebook from PreprocessLib.ipynb


In [2]:
import spacy
#processes file for frequency analysis, optionally tokenizes
def process_file(a_file, tokenize=True):
    string_without_line_breaks = ""
    for line in a_file:
        stripped_line = line.rstrip() + " "
        string_without_line_breaks += stripped_line
    a_file.close()
    sentences = string_without_line_breaks
    
    
   
    if (tokenize == False):
        cleanedFile = cleanText(sentences, tokenize)
        return cleanedFile
    cleanedFile = cleanText(sentences) #cleanText is found in PreprocessLib.ipynb
        
    output_doc = []
    nlp = spacy.load("el_core_news_sm")

    for sentence in cleanedFile:
        temp = []
        doc = nlp(sentence)
        for token in doc:
            temp.append(token.text)
        output_doc.append(temp)
        output_doc = removeSingleLists(output_doc)
    return output_doc

In [3]:
#returns file in string format
def stringify(a_file):
    string_without_line_breaks = ""
    for line in a_file:
        stripped_line = line.rstrip() + " "
        string_without_line_breaks += stripped_line
    a_file.close()
    sentences = string_without_line_breaks
    return sentences

In [4]:
longFile = open("./other-sources/Longus.txt", "r", encoding="utf8")
markFile = open("./other-sources/Mark.txt", "r", encoding="utf8")
mathFile = open("./other-sources/Mathew.txt", "r", encoding="utf8")
ignatFile = open("./other-sources/Ignatius-4.txt", "r", encoding="utf8")
testFile = open("./other-sources/Ignatius-3books.txt", "r", encoding="utf8")
longFreq = process_file(longFile, False)
markFreq = process_file(markFile, False)
mathFreq = process_file(mathFile, False)
ignatFreq = process_file(ignatFile, False)
testFreq = process_file(testFile, False)

longFile = open("./other-sources/Longus.txt", "r", encoding="utf8")
markFile = open("./other-sources/Mark.txt", "r", encoding="utf8")
mathFile = open("./other-sources/Mathew.txt", "r", encoding="utf8")
ignatFile = open("./other-sources/Ignatius-4.txt", "r", encoding="utf8")
testFile = open("./other-sources/Ignatius-3books.txt", "r", encoding="utf8")

#sentence tokenizes the documents
long = stringify(longFile)
mark = stringify(markFile)
math = stringify(mathFile)
ignat = stringify(ignatFile)
test = stringify(testFile)
from nltk import tokenize
longSent = tokenize.sent_tokenize(long)
markSent = tokenize.sent_tokenize(mark)
mathSent = tokenize.sent_tokenize(math)
ignatSent = tokenize.sent_tokenize(ignat)
testSent = tokenize.sent_tokenize(test)

In [5]:
print(len(longFreq.split()))
print(len(markFreq.split()))
print(len(mathFreq.split()))
print(len(ignatFreq.split()))

19826
11282
18349
3969


In [6]:
#uses a g-test, useful for finding differences in proportions
#returns signifcance of G-Test
def isSignificant(obA, exA, obB, exB):
    import numpy as np
    from scipy.stats import chi2_contingency
    obs = np.array([[obA, exA-obA], [obB, exB-obB]])
    p = chi2_contingency(obs, lambda_="log-likelihood")
    pVal = p[1] 
    if (pVal < 0.05):
        return True
    else:
        return False

In [7]:
import re
from operator import itemgetter
import numpy as np
from scipy.stats import chi2_contingency
#returns list of instances and count in sorted order from most to least common
def mostCommon(instances):
    return sorted(sorted(Counter(instances).items(), key=itemgetter(0)), key=itemgetter(1), reverse=True)

#returns frequency of word in txt
def getFreq(word, txt):
    return txt.count(word)

#analyzes frequency of the words in the document
#returns most common words of each document and count of significantly different word frequencies
# and each iteration counts which authors words was most similar, then returns the list
def analyzeFreq():
    from scipy import stats
    from collections import Counter
    longList = re.sub("[^\w]", " ",  longFreq).split()
    longCounts = Counter(longList)
    markList = re.sub("[^\w]", " ",  markFreq).split()
    markCounts = Counter(markList)
    mathList = re.sub("[^\w]", " ",  mathFreq).split()
    mathCounts = Counter(mathList)
    ignatList = re.sub("[^\w]", " ",  ignatFreq).split()
    ignatCounts = Counter(ignatList)
    testList = re.sub("[^\w]", " ",  testFreq).split()
    testCounts = Counter(testList)

    print(longCounts.most_common(10))
    print(markCounts.most_common(10))
    print(mathCounts.most_common(10))
    print(ignatCounts.most_common(10))
    print(testCounts.most_common(10))
    
    countList = []
    L = 0
    Ma = 0
    Mr = 0
    Ig = 0
    prevWords = []
    wordCount = 0
    for word in testCounts:
        label = 'notinany'
        minDiff = 99999 
        count = 0
        testWordFreq = getFreq(word, testFreq)
        if (word in longCounts):
            tempFreq = getFreq(word, longFreq)
            if (isSignificant(tempFreq, len(longList), testWordFreq, len(testList))):
                L += 1
            tempDiff = abs(longCounts[word] - testCounts[word])
            count += 1
            if (tempDiff < minDiff):
                label = "Longus"
                minDiff = tempDiff
        if (word in markCounts):
            tempFreq = getFreq(word, markFreq)
            if (isSignificant(tempFreq, len(markList), testWordFreq, len(testList))):     
                Mr += 1
            tempDiff = abs(markCounts[word] - testCounts[word])
            count += 1
            if (tempDiff < minDiff):
                label = "Mark"
                minDiff = tempDiff
        if (word in mathCounts):
            tempFreq = getFreq(word, mathFreq)
            if (isSignificant(tempFreq, len(mathList), testWordFreq, len(testList))):     
                Ma += 1
            tempDiff = abs(mathCounts[word] - testCounts[word])
            count += 1
            if (tempDiff < minDiff):
                label = "Mathew"
                minDiff = tempDiff
        if (word in ignatCounts):
            tempFreq = getFreq(word, ignatFreq)
            if (isSignificant(tempFreq, len(ignatList), testWordFreq, len(testList))):     
                Ig += 1
            tempDiff = abs(ignatCounts[word] - testCounts[word])
            count += 1
            if (tempDiff < minDiff):
                label = "Ignatius"
                minDiff = tempDiff
        if (label != 'notinany' and count > 3):
            countList.append(label)
            wordCount += 1
        prevWords.append(word)
    from collections import Counter
    from collections import OrderedDict
    list = Counter(countList)
    print(OrderedDict([(i, str(round(count / sum(list.values()) * 100.0, 3)) + '%') for i, count in list.most_common()]))
    print(L, Ma, Mr, Ig)
    print(L/(wordCount), Ma/(wordCount), Mr/(wordCount), Ig/(wordCount))
    
    

In [12]:
analyzeFreq() # no stop words

[('τὰς', 152), ('ἦν', 98), ('Δάφνις', 96), ('Καὶ', 71), ('Χλόη', 64), ('Χλόην', 63), ('ταῖς', 60), ('Δάφνιν', 58), ('τοῦτο', 46), ('αὐτοῖς', 46)]
[('καὶ', 407), ('αὐτοῦ', 173), ('αὐτῶ', 121), ('αὐτοῖς', 120), ('αὐτὸν', 117), ('τῶ', 77), ('ἵνα', 64), ('λέγει', 62), ('αὐτόν', 61), ('εἶπεν', 59)]
[('καὶ', 276), ('αὐτοῦ', 266), ('αὐτῶ', 170), ('τῶ', 149), ('εἶπεν', 119), ('ἰησοῦς', 111), ('ὑμῖν', 107), ('αὐτοῖς', 103), ('αὐτῶν', 100), ('σου', 98)]
[('θεοῦ', 68), ('Ἰησοῦ', 39), ('ἵνα', 36), ('Χριστοῦ', 34), ('ὑμῶν', 34), ('μοι', 34), ('αὐτοῦ', 27), ('ὑμῖν', 25), ('με', 23), ('ὑμᾶς', 22)]
[('θεοῦ', 54), ('Ἰησοῦ', 50), ('ὑμῶν', 47), ('ὑμᾶς', 35), ('αὐτοῦ', 32), ('Χριστοῦ', 30), ('ἵνα', 28), ('ὑμῖν', 27), ('ἐστιν', 27), ('Χριστῷ', 20)]
OrderedDict([('Ignatius', '42.657%'), ('Longus', '27.972%'), ('Mark', '23.077%'), ('Mathew', '6.294%')])
71 62 39 10
0.4965034965034965 0.43356643356643354 0.2727272727272727 0.06993006993006994


In [8]:
analyzeFreq() #stop words

[('καὶ', 1378), ('δὲ', 547), ('τὴν', 320), ('μὲν', 278), ('τὸν', 225), ('τῶν', 224), ('τὸ', 209), ('ὁ', 206), ('τὰ', 180), ('τῆς', 179)]
[('καὶ', 1082), ('ὁ', 236), ('αὐτοῦ', 173), ('εἰς', 168), ('δὲ', 153), ('τὸν', 149), ('ἐν', 135), ('τοῦ', 132), ('τὸ', 130), ('τὴν', 126)]
[('καὶ', 1175), ('ὁ', 493), ('δὲ', 471), ('τοῦ', 294), ('ἐν', 293), ('αὐτοῦ', 266), ('τὸ', 227), ('οἱ', 224), ('τὸν', 221), ('εἰς', 218)]
[('καὶ', 205), ('ἐν', 100), ('θεοῦ', 68), ('τὸ', 60), ('τοῦ', 57), ('δὲ', 54), ('εἰς', 52), ('ὡς', 45), ('Ἰησοῦ', 39), ('ἵνα', 36)]
[('καὶ', 185), ('ἐν', 135), ('τοῦ', 66), ('θεοῦ', 54), ('Ἰησοῦ', 50), ('τὸ', 50), ('εἰς', 47), ('ὑμῶν', 47), ('τῷ', 41), ('γὰρ', 38)]
OrderedDict([('Ignatius', '51.042%'), ('Longus', '22.396%'), ('Mark', '20.312%'), ('Mathew', '6.25%')])
91 90 66 13
0.4739583333333333 0.46875 0.34375 0.06770833333333333


In [68]:
#gets all similar words of the texts, excluding puncuation, then returns list
def getAllSimWords():
    simWords = []
    for word in testList:
        puncuation = [",", "."]
        if (word in longList and word in markList and word in mathList and word in ignatList and word in testList and word not in puncuation):
            simWords.append(word)
    return simWords

In [22]:
from scipy import stats
from collections import Counter
import re
#creates counter of each word in the documents
longList = re.sub("[^\w]", " ",  longFreq).split()
longCounts = Counter(longList)
markList = re.sub("[^\w]", " ",  markFreq).split()
markCounts = Counter(markList)
mathList = re.sub("[^\w]", " ",  mathFreq).split()
mathCounts = Counter(mathList)
ignatList = re.sub("[^\w]", " ",  ignatFreq).split()
ignatCounts = Counter(ignatList)
testList = re.sub("[^\w]", " ",  testFreq).split()
testCounts = Counter(testList)
print(longCounts.most_common(10))
print(markCounts.most_common(10))
print(mathCounts.most_common(10))
print(ignatCounts.most_common(10))
print(testCounts.most_common(10))

[('τὰς', 152), ('ἦν', 98), ('Δάφνις', 96), ('Καὶ', 71), ('Χλόη', 64), ('Χλόην', 63), ('ταῖς', 60), ('Δάφνιν', 58), ('τοῦτο', 46), ('αὐτοῖς', 46)]
[('καὶ', 407), ('αὐτοῦ', 173), ('αὐτῶ', 121), ('αὐτοῖς', 120), ('αὐτὸν', 117), ('τῶ', 77), ('ἵνα', 64), ('λέγει', 62), ('αὐτόν', 61), ('εἶπεν', 59)]
[('καὶ', 276), ('αὐτοῦ', 266), ('αὐτῶ', 170), ('τῶ', 149), ('εἶπεν', 119), ('ἰησοῦς', 111), ('ὑμῖν', 107), ('αὐτοῖς', 103), ('αὐτῶν', 100), ('σου', 98)]
[('θεοῦ', 68), ('Ἰησοῦ', 39), ('ἵνα', 36), ('Χριστοῦ', 34), ('ὑμῶν', 34), ('μοι', 34), ('αὐτοῦ', 27), ('ὑμῖν', 25), ('με', 23), ('ὑμᾶς', 22)]
[('θεοῦ', 54), ('Ἰησοῦ', 50), ('ὑμῶν', 47), ('ὑμᾶς', 35), ('αὐτοῦ', 32), ('Χριστοῦ', 30), ('ἵνα', 28), ('ὑμῖν', 27), ('ἐστιν', 27), ('Χριστῷ', 20)]


In [69]:
#generates all shared words between the documents
commonWordList = getAllSimWords()

In [88]:
import numpy as np
#computes cosine similarity of vector A and vector B
def cosineSim(A, B):
    sim = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
    if np.isnan(np.sum(sim)):
        return 0
    return sim

In [7]:
count = longCounts & markCounts & mathCounts & ignatCounts & testCounts 

In [21]:
#https://github.com/alpheios-project/arethusa-configs/blob/master/configs/arethusa.morph/gr_attributes2.json
# The possible tags for each item of morphology
  #  pos_tags = ('l', 'n', 'a', 'r', 'c', 'i', 'p', 'v', 'd', 'm', 'g', 'u')
    #l - article
    #n - noun
    #a - adjective
    #r - adposition
    #c - conjugation
    #i - interjection
    #p - pronoun
    #v - verb
    #d - adverb
    #m - numeral
    #u - puncuation
    #x - irregular
    #g - participle
    
    
  #  person_tags = ('1', '2', '3')

  #  number_tags = ('s', 'p', 'd')
    
  # tense_tags = ('p', 'i', 'r', 'l', 't', 'f', 'a')       
    #p - present
    #i - imperfect
    #r - perfect
    #l - plusquamperfect
    #t - future perfect
    #f - future
    #a - aorist
    
  #  mood_tags = ('i', 's', 'n', 'm', 'p', 'o')
    #i - indicative
    #s - subjunctive
    #n - infinitive
    #m - imperative
    #p - participle
    #o - optative
    
  #  voice_tags = ('a', 'p', 'm', 'e')
    #a - active
    #p - passive
    #m - middle
    #e - medio-passive

  #  gender_tags = ('m', 'f', 'n')
    
  #  case_tags = ('n', 'g', 'd', 'a', 'v')
    
  #  degree_tags = ('p', 'c', 's')
    #p - positive
    #c - comparative
    #s - superlative
    
#testing POS tagger
from angel import tag
greek_string = 'Τοιαῦτα δὲ αὐτῶν παιζόντων τοιάνδε σπουδὴν Ἔρως ἀνέπλασε.'

results = tag(greek_string)
print(results)

Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood models loading...
Voice models loading...
Gender models loading...
Case models loading...
Degree models loading...
Pre-processing text...
Text and punctuation split into 9 individual tokens.
Angel's looking at each word by itself...
Reconsidering tags...
Studying each word in light of its context...
(('Τοιαῦτα', 'p-p---na-'), ('δὲ', 'd--------'), ('αὐτῶν', 'p-p---mg-'), ('παιζόντων', 'v-pppamg-'), ('τοιάνδε', 'a-s---fa-'), ('σπουδὴν', 'n-s---fa-'), ('Ἔρως', 'n-s---mn-'), ('ἀνέπλασε', 'v3saia---'), ('.', 'u--------'))


In [98]:
#POS tagging for each document (this took forever)
from angel import tag
longTags = tag(long)
markTags = tag(mark)
mathTags = tag(math)
ignatTags = tag(ignat)
testTags = tag(test)

Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood models loading...
Voice models loading...
Gender models loading...
Case models loading...
Degree models loading...
Pre-processing text...
Text and punctuation split into 22671 individual tokens.
Angel's looking at each word by itself...
Reconsidering tags...
Studying each word in light of its context...
Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood models loading...
Voice models loading...
Gender models loading...
Case models loading...
Degree models loading...
Pre-processing text...
Text and punctuation split into 12741 individual tokens.
Angel's looking at each word by itself...
Reconsidering tags...
Studying each word in light of its context...
Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood 

In [217]:
#takes in  list of all tagLists "tags", a word to look for "testCase", and the index of the POS tag of interest
#returns frequencies of testCase in each document, and G-test result of the frequencies with the test frequencies
def freqOf(tags, testCase, index):
    results = []
    for tagList in tags:
        counter = 0
        for word in tagList:
            if (word[1][index] == testCase):
                counter += 1
        results.append(counter)
    
    print(results)
    i = 0
    for case in results[:-1]:
        print("Testing index ", i, " with test:")
        print("Frequencies: ", round(case/len(tags[i]), 5), " ", round(results[4]/len(tags[4]),5))
        if (case == 0):
            case = 0.000000000000000000001 #if case stays 0, there is an error with the g-test
        if (isSignificant(case, len(tags[i]), results[4], len(tags[4]))): #index 4 is index of test document
            print("Different")
        else:
            print("Not Different")
        print('\n')
        i += 1

In [138]:
#testing frequency of optative mood usage 
tags = [longTags, markTags, mathTags, ignatTags, testTags]
indexMood = 4
freqOf(tags, "o", indexMood)

[45, 1, 0, 6, 6]
Testing index  0  with test:
Frequencies:  0.00283   0.00188
Not Different


Testing index  1  with test:
Frequencies:  0.00011   0.00188
Different


Testing index  2  with test:
Frequencies:  0.0   0.00188
Different


Testing index  3  with test:
Frequencies:  0.00174   0.00188
Not Different




In [143]:
#testing frequency of future perfect tense usage
indexTense = 3
freqOf(tags, "t", indexTense)

[2, 0, 0, 0, 0]
Testing index  0  with test:
Frequencies:  0.00013   0.0
Not Different


Testing index  1  with test:
Frequencies:  0.0   0.0
Not Different


Testing index  2  with test:
Frequencies:  0.0   0.0
Not Different


Testing index  3  with test:
Frequencies:  0.0   0.0
Not Different




In [144]:
#testing frequency of pluperfect tense usage
freqOf(tags, "l", indexTense)

[54, 8, 7, 0, 0]
Testing index  0  with test:
Frequencies:  0.0034   0.0
Different


Testing index  1  with test:
Frequencies:  0.00085   0.0
Not Different


Testing index  2  with test:
Frequencies:  0.00047   0.0
Not Different


Testing index  3  with test:
Frequencies:  0.0   0.0
Not Different




In [349]:
#takes in a sentence, and splits the sentence
#returns the split sentence as a tuple
def splitSentence(sentence, i):
    first = ""
    second = ""
    found = False
    for word in sentence:
        splitWords = ["καί", "δέ", "ἀλλά", "τε", "ἄρα", "διό"]
        if (word in splitWords and len(first) > 2):
            found = True
        if (found):
            second += " " + word
        else:
            first += " " + word
    return(first, second)

In [380]:
#takes in a tagList and returns counts of participles before or after main verb
def verbCheck(tagList):
    leftCount = 0
    rightCount = 0
    mainVerbFound = False
    for word in tagList:
        if ((word[1][4] == 'i' or word[1][4] == 'm') and word[1][0] == 'v' and mainVerbFound == False):
            mainVerbFound = True
            leftCount += 1
        else:
            rightCount += 1
    return (leftCount, rightCount)

In [441]:
#this is an algorithm to iterate a corpus, while also 'keeping up' with the tag list. 
#This is necessary because tagging sentence by sentence is VERY inefficient and takes too long,
#so figuring out a way to iterate the corpus, while keeping on the same word as the tag list is important
#this algorithm is used below in particpleAnalysis
import string
j = 0
table = str.maketrans(dict.fromkeys(string.punctuation))
splitWords = ["καί", "δέ", "ἀλλά", "τε", "ἄρα", "διό"]
for sent in testSent:
    sentenceSplit = sent.split()
    for word in sentenceSplit:
        if (testTags[j][0] in string.punctuation):
            j += 1
        testWord = word.translate(table)
        line = testWord
        testWord = ''.join( c for c in line if  c not in '’᾿·' ) #to fix bug with greek punctuation
        testWord2 = testTags[j][0].translate(table)
        line = testWord2
        testWord2 = ''.join( c for c in line if  c not in '’᾿·' )
        if (len(testWord2) < 1): #to fix a bug with multiple spaces present in word
            j += 2
        if (testWord == testWord2):
            j += 1


In [426]:
#takes in a text and a tag list, returns count of participles before and after main verb
def participleAnalysis(txt, tagList):
    import string
    j = 0
    leftCount = 0
    rightCount = 0
    table = str.maketrans(dict.fromkeys(string.punctuation)) 
    for sent in txt:
        sentenceSplit = sent.split()
       # print("sent: ", sentenceSplit)
        firstHalf = []
        secondHalf = []
        enterSecond = False
        for word in sentenceSplit:
            if (longTags[j][0] in string.punctuation):
                j += 1
            testWord = word.translate(table)
            line = testWord
            testWord = ''.join( c for c in line if  c not in '’᾿·' ) #to fix a bug with greek punctuation
            testWord2 = tagList[j][0].translate(table)
            line = testWord2
            testWord2 = ''.join( c for c in line if  c not in '’᾿·' )
            if (len(testWord2) < 1): #to fix a bug with multiple spaces present in word
                j += 2
            if (testWord == testWord2):

                firstHalf.append(tagList[j])
                if (testWord in splitWords and len(firstHalf) > 3 and enterSecond == False):
                    enterSecond = True
                j += 1
            if (enterSecond == True):
                secondHalf.append(tagList[j])
        if (enterSecond == True):
            count = verbCheck(firstHalf)
            leftCount += count[0]
            rightCount += count[1]
            count = verbCheck(secondHalf)
            leftCount += count[0]
            rightCount += count[1]
        if (enterSecond == False):
            count = verbCheck(firstHalf)
            leftCount += count[0]
            rightCount += count[1]
    print(leftCount)
    print(rightCount)

In [429]:
#Longus
participleAnalysis(longSent, longTags)

762
21335


In [430]:
#Mark
participleAnalysis(markSent, markTags)

64
1131


In [431]:
#Mathew
participleAnalysis(mathSent, mathTags)

103
2252


In [432]:
#Ignatius
participleAnalysis(ignatSent, ignatTags)

181
2883


In [433]:
participleAnalysis(testSent, testTags)

143
3020


In [451]:
isSignificant(762, 21335, 143, 3020)

True

In [452]:
isSignificant(64, 1131, 143, 3020)

False

In [453]:
isSignificant(103, 2252, 143, 3020)

False

In [454]:
isSignificant(181, 2883, 143, 3020)

True